In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from cartopy import crs as ccrs 
import cartopy.feature as cfeature

## LPJ

### Daily NPP

In [ ]:
lpj_path = "/discover/nobackup/projects/GHGC/LPJ_collaborations/NRT_carbon_budget_const_lu/20250521/S2_RESP_ACCLIM/ncdf_outputs"

In [ ]:
lpj_dnpp = f"{lpj_path}/ERA5_S2_RESP_ACCLIM_dnpp.nc"

In [ ]:
ds_dnpp = xr.open_dataset(lpj_dnpp)
ds_dnpp

### Daily Rh

In [ ]:
lpj_drh = f"{lpj_path}/ERA5_S2_RESP_ACCLIM_drh.nc"

In [ ]:
ds_drh = xr.open_dataset(lpj_drh)
ds_drh

### NEE = RH - NPP

In [ ]:
ds_lpj_combined = xr.merge([ds_drh, ds_dnpp])
ds_lpj_combined

In [ ]:
xr.align(ds_drh, ds_dnpp, join='exact') # Check if this spits an error

In [ ]:
# This crashes the node
# ds_combined['dnee'] = ds_combined['drh'] - ds_combined['dnpp']

## Plotting tests

In [ ]:
ds_sel = ds["dnpp"].isel(time=0)

In [ ]:
ds_sel

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8), subplot_kw= {'projection': proj})
ds_sel.plot(ax=ax, transform=ccrs.PlateCarree())

In [ ]:
plt.show()

In [ ]:
ds_sel.plot()

## MiCASA

In [ ]:
micasa_path = "micasa_virtualized/vstore.parquet"

In [ ]:
ds_mi = xr.open_dataset(f"reference::{micasa_path}", engine="zarr", consolidated=False)
ds_mi

## Compare and plot

In [ ]:
# set map proj
proj=ccrs.PlateCarree()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8), subplot_kw= {'projection': proj})
plot = ds_mi["NPP"].isel(time=0).plot(ax=ax,
                                      cbar_kwargs=dict(shrink=0.6)
                                     )
# plt.suptitle("MiCASA test")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8), subplot_kw= {'projection': proj})
plot = ds_lpj_combined["dnpp"].isel(time=0).plot(ax=ax, 
                                                cbar_kwargs=dict(shrink=0.6))

# These are in different units and in different resolutions

## Micasa only starts at Jan 2001

In [ ]:
ds_lpj_sel = ds_lpj_combined.sel(time=slice("2001", None))
ds_lpj_sel

### 6 days missing? LPJ does NOT include leap years?? I'm going to freak out

### I'm just going to have to skip those days

## First fix resolution

In [ ]:
# ds_mi.interp_like(ds_lpj_sel)
# This crashes the node. I need to use dask or something. A task for tomorrow.....